<a href="https://colab.research.google.com/github/ldk7024/Deep_Learning_Study/blob/main/21_07_20_%EA%B0%9C%2C_%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%82%AC%EC%A7%84_%EC%98%88%EC%B8%A1_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- NPZ 파일 로딩

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = np.load('/content/drive/MyDrive/Colab Notebooks/빅데이터4차(딥러닝)/data/cats_and_dogs.npz')
data

In [3]:
len(data)
# X_train, X_test, y_train, y_test 총 4가지로 구성되어 있음

4

In [4]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2000, 224, 224, 3)
(1000, 224, 224, 3)
(2000,)
(1000,)
